<a href="https://colab.research.google.com/github/OlhaZahrebelna/ab_test_pl/blob/main/AB_test_PL_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AB test

Musimy przeanalizować dane z testu A/B w popularnej grze [Cookie Cats] (https://www.facebook.com/cookiecatsgame). est to klasyczna gra logiczna w stylu „połącz trzy”, w której gracz musi łączyć płytki tego samego koloru, aby oczyścić planszę i wygrać poziom. Na planszy znajdują się także śpiewające kotki :)

Podczas rozgrywki gracze napotykają bramy, które zmuszają ich do oczekiwania przez pewien czas, zanim będą mogli kontynuować grę lub dokonać zakupu w aplikacji. W tym zadaniu przeanalizujemy wyniki testu A/B, w którym pierwsze bramy w grze Cookie Cats zostały przeniesione z poziomu 30 na poziom 40. W szczególności przeanalizujemy wpływ tej zmiany na retencję graczy (retention). Chcemy zrozumieć, czy przesunięcie bram o 10 poziomów później w jakikolwiek sposób wpłynęło na to, czy użytkownicy przestają grać wcześniej lub później w kontekście liczby dni od momentu zainstalowania gry.

Będziemy pracować z danymi z pliku `cookie_cats.csv`. Zmienne w danych są następujące:

userid – unikalny numer identyfikujący każdego gracza.
version – informacja, czy gracz trafił do grupy kontrolnej (gate_30 – bramy na 30. poziomie) czy do grupy testowej (gate_40 – bramy na 40. poziomie).
sum_gamerounds – liczba rund gry rozegranych przez gracza w ciągu pierwszego tygodnia po instalacji.
retention_1 – czy gracz wrócił i zaczął grać dzień po instalacji gry?
retention_7 – czy gracz wrócił i zaczął grać siedem dni po instalacji gry?
Podczas instalacji gry gracze byli losowo przypisywani do grupy gate_30 lub gate_40.

1. Wczytaj dane z testu A/B do zmiennej df i wyświetl średnią wartość wskaźnika retention_7 (utrzymanie po 7 dniach) dla poszczególnych wersji gry.
Sformułuj hipotezę: która wersja gry zapewnia lepsze utrzymanie graczy po 7 dniach od instalacji?

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [2]:
df = pd.read_csv('/content/cookie_cats.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


Hipoteza zerowa (H0): Utrzymanie graczy po 7 dniach jest takie samo dla obu wersji gry (gate_30 i gate_40).
Hipoteza alternatywna (H1): Utrzymanie graczy po 7 dniach różni się w zależności od wersji gry; jedna z wersji zapewnia lepsze utrzymanie.

In [4]:
df.groupby('version')['retention_7'].mean()

,retention_7
version,
gate_30,0.190201
gate_40,0.182000


Utrzymanie graczy jest nieco wyższe w wersji gate_30.

2. Sprawdź za pomocą testu z, podobnie jak w przykładzie na wykładzie, czy któraś z wersji gry daje lepszy wynik retention_7 na poziomie istotności 0,05. Oblicz również przedziały ufności dla dwóch próbek. Wynik powinien zostać wyświetlony w następującym formacie:
```
z statistic: ...
p-value: ...
Przedział ufności 95% dla grupy control: [..., ...]
Przedział ufności 95% dla grupy treatment: [..., ...]
```
Zamiast ... – obliczone wartości. Wnioski powinny odpowiadać na dwa pytania:

Czy istnieje statystycznie istotna różnica między zachowaniem użytkowników w różnych wersjach gry?
Czy przedziały ufności dla utrzymania użytkowników w różnych wersjach gry się pokrywają? Co to oznacza?

In [5]:
df['version'].value_counts()

,count
version,
gate_40,45489
gate_30,44700


In [6]:
session_control = df['userid'].value_counts(ascending=False)
multi_users = session_control[session_control > 1].count()
multi_users

0

In [7]:
# Podziel dane na grupę kontrolną i testową
group_30 = df[df['version'] == 'gate_30']
group_40 = df[df['version'] == 'gate_40']

In [8]:
success_30 = group_30['retention_7'].sum()
n_30 = len(group_30)

success_40 = group_40['retention_7'].sum()
n_40 = len(group_40)

In [9]:
z_stat, p_value = proportions_ztest([success_30, success_40], [n_30, n_40])


In [10]:
# Przedziały ufności dla dwóch próbek
conf_int_30 = proportion_confint(success_30, n_30, alpha=0.05, method='normal')
conf_int_40 = proportion_confint(success_40, n_40, alpha=0.05, method='normal')


In [11]:
print(f"z statistic: {z_stat}")
print(f"p-value: {p_value}")
print(f"Przedział ufności 95% dla grupy gate_30: {conf_int_30}")
print(f"Przedział ufności 95% dla grupy gate_40: {conf_int_40}")

z statistic: 3.164358912748191
p-value: 0.001554249975614329
Przedział ufności 95% dla grupy gate_30: (0.18656311652199903, 0.19383956804175934)
Przedział ufności 95% dla grupy gate_40: (0.17845430073314686, 0.18554578720019968)


Odrzucamy hipotezę zerową (Hipoteza zerowa (H0): Utrzymanie graczy po 7 dniach jest takie same dla obu wersji gry (gate_30 i gate_40)). Istnieje statystycznie istotna różnica w utrzymaniu, ponieważ p-value jest mniejsze niż 0,05. Przedziały dla grup gate_30 i gate_40 nie nakładają się. Dla gate_30 przedział mieści się w zakresie (0.1866, 0.1938), a dla gate_40 – (0.1785, 0.1855). To potwierdza, że utrzymanie w grupie kontrolnej (gate_30) jest wyższe niż w grupie testowej (gate_40). Chociaż jest to dość dziwne.

3. Istnieje jeszcze jeden typ testów, który jest stosowany do metryk binarnych, takich jak "czy użytkownik wykona akcję, czy nie" – test Chi-kwadrat. Ma on inne hipotezy H0 i H1 w porównaniu do testów z- i t-testów. Ponadto ten test można stosować, gdy mamy więcej niż dwie grupy badawcze, czyli gdy nie mamy testu A/B, ale np. A/B/C/D.

W testach z- i t (które różnią się tym, że w pierwszym nie znamy wariancji w populacji, ale jeśli mamy dużą próbkę danych, to te dwa testy dają bardzo podobne wyniki) sprawdzamy, czy istnieje różnica w średnich wynikach pomiędzy grupami użytkowników. Zadanie: wykonaj test Chi-kwadrat na poziomie istotności 5%, aby sprawdzić, czy istnieje zależność między wersją gry a tym, czy gracz powróci na 7. dzień po zainstalowaniu gry. Hipotezy są następujące:

H0: wartość retention_7 nie zależy od wersji gry
H1: istnieje zależność między wersją gry a wartością retention_7

Wykonaj test, wyświetl p-wartość i dokonaj wniosków. W teście Chi-kwadrat sprawdzamy, czy istnieje związek między grupą użytkowników a tym, czy wykona on interesującą nas akcję. To jakby badanie tego samego, ale nieco różnymi metodami. Aby zweryfikować wynik, można przeprowadzić kilka testów (szczególnie, jeśli jeden daje nieprzekonujący wynik, np. p-wartość 0,07 – niby fail to reject H0 na poziomie 5%, ale ciekawe, co powiedzą inne testy), więc przeprowadzimy test Chi-kwadrat i porównamy jego wynik z testem z.

In [12]:
table = pd.crosstab(df['version'], df['retention_7'])
table

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [13]:
chi2_stat, p_value, dof, expected = stats.chi2_contingency(table)
print(f"p-value: {p_value}")


p-value: 0.0016005742679058301


In [14]:
print(f'χ² statistic: {chi2_stat}')
print(f'Degrees of freedom: {dof}')
print('Expected frequencies:')
print(expected)

χ² statistic: 9.959086799559167
Degrees of freedom: 1
Expected frequencies:
[[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]


p-wartość jest mniejsza niż 0,05, wersja gry wpływa na retention_7.
Odrzucamy hipotezę zerową (H₀): wartość retention_7 nie zależy od wersji gry.